**Week 3 Part 1**

**1. Scraping the table data from wikipedia page using BeautifulSoup and creating a dataframe with only columns**

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(URL)
soup = BeautifulSoup(response.text,"html.parser")

table = soup.find('table',{'class':'wikitable sortable'}).tbody

rows = table.find_all('tr')

columns = [v.text.replace('\n','') for v in rows[0].find_all('th')]

df = pd.DataFrame(columns=columns)

In [2]:
df.rename(columns={"Postcode":"PostalCode"},inplace=True)
df.columns

Index(['PostalCode', 'Borough', 'Neighbourhood'], dtype='object')

**2. Reading the row data from table rows and appending it into the df dataframe.**

In [3]:
df.head()

,PostalCode,Borough,Neighbourhood


In [4]:
for i in range(1,len(rows)):
    tds = rows[i].find_all('td')
    if tds[1].text != 'Not assigned':
            if (tds[2].text.replace('\n','') == 'Not assigned'):
                values = [tds[0].text,tds[1].text,tds[1].text]
            else :   
                values = [tds[0].text,tds[1].text,tds[2].text.replace('\n','')]
            df = df.append(pd.Series(values,index=columns),ignore_index=True)
    else:
        values = [td.text.replace('\n','') for td in tds]
#df.to_csv(r'C:\Users\dsapre\Documents\Python Scripts\Canada1.csv',index=False)

In [5]:
df.head()

,PostalCode,Borough,Neighbourhood,Postcode
0,NaN,North York,Parkwoods,M3A
1,NaN,North York,Victoria Village,M4A
2,NaN,Downtown Toronto,Harbourfront,M5A
3,NaN,Downtown Toronto,Regent Park,M5A
4,NaN,North York,Lawrence Heights,M6A


**3. Groupby Postcode and Borough and then join by Neighborhood so that there is only row per Postcode and Borough**

In [6]:
df.drop(['PostalCode'],axis=1,inplace=True)

In [7]:
df.rename(columns={"Postcode":"PostalCode"},inplace=True)
df.columns

Index(['Borough', 'Neighbourhood', 'PostalCode'], dtype='object')

In [8]:
df = df[['PostalCode','Borough','Neighbourhood']]
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [9]:
df_grouped = df.groupby(['PostalCode','Borough'],as_index=False).agg(lambda group: ','.join(group))
#df.to_csv(r'C:\Users\dsapre\Documents\Python Scripts\Canada1.csv',index=True)

In [10]:
df_grouped.columns

Index(['PostalCode', 'Borough', 'Neighbourhood'], dtype='object')

In [11]:
df_grouped.shape

(103, 3)